# Project Cryosphere by Matthias Göbel and Philipp Gregor
## Part 1

In [ ]:
# imports and defaults
import pandas as pd  
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 14
import seaborn as sns
sns.set_style('ticks')
sns.set_context('talk')

In [ ]:
# read the data
df = pd.read_csv('data/aws_data_zhadang_localtime.csv', index_col=0, parse_dates=True)

## 1 Radiative Fluxes

Calculate the radiative fluxes from the data.

Look at manual.

## 2 Turbulent Fluxes

## 3 Residual Flux $F$ and melt energy $Q_M$

## 4 Mass balance

## 5 An attempt of validation